In [1]:
import pandas as pd

### How to merge DataFrames in pandas?

- `df1.append(df2)`: stacking vertically
- `pd.concat([df1, df2])`:
    - stacking many horizontally or vertically
    - simple inner/outer joins on Indexes
- `df1.join(df2)`: inner/outer/left/right joins on Indexes
- `pd.merge(df1, df2)`: many joins on multiple columns

In [2]:
# movies dataset

movie_cols = ['movie_id', 'title']
movies = pd.read_csv('http://bit.ly/movieitems', sep='|', header=None, names=movie_cols, usecols=[0, 1])
movies.head()

,movie_id,title
0,1,Toy Story (1995)
1,2,GoldenEye (1995)
2,3,Four Rooms (1995)
3,4,Get Shorty (1995)
4,5,Copycat (1995)


In [3]:
movies.shape

(1682, 2)

In [4]:
# to check the number of unique values

movies.movie_id.nunique()

1682

In [5]:
# ratings dataset

rating_cols = ['user_id', 'movie_id', 'rating', 'timestamp']
ratings = pd.read_csv('http://bit.ly/movielensdata', sep='\t', header=None, names=rating_cols)
ratings.head()

,user_id,movie_id,rating,timestamp
0,196,242,3,881250949
1,186,302,3,891717742
2,22,377,1,878887116
3,244,51,2,880606923
4,166,346,1,886397596


In [6]:
ratings.shape

(100000, 4)

In [7]:
ratings.movie_id.nunique()

1682

In [8]:
# To see the top 5 ratings for the 'movie_id 1'

ratings.loc[ratings.movie_id == 1, :].head()

,user_id,movie_id,rating,timestamp
24,308,1,4,887736532
454,287,1,5,875334088
957,148,1,4,877019411
971,280,1,4,891700426
1324,66,1,3,883601324


**Merging Movies and Ratings**

Let's pretend that you want to examine the ratings DataFrame, but you want to know the **title** of each movie rather than its **movie_id**. The best way to accomplish this objective is by "joining" (or "merging") the DataFrames using the Pandas `merge` function:

In [9]:
movies.columns

Index(['movie_id', 'title'], dtype='object')

In [10]:
ratings.columns

Index(['user_id', 'movie_id', 'rating', 'timestamp'], dtype='object')

In [11]:
# Now merging both the dataframes and creating a new one

movie_ratings = pd.merge(movies, ratings)
movie_ratings.head()

,movie_id,title,user_id,rating,timestamp
0,1,Toy Story (1995),308,4,887736532
1,1,Toy Story (1995),287,5,875334088
2,1,Toy Story (1995),148,4,877019411
3,1,Toy Story (1995),280,4,891700426
4,1,Toy Story (1995),66,3,883601324


In [12]:
movie_ratings.shape

(100000, 5)

In [13]:
movie_ratings.columns

Index(['movie_id', 'title', 'user_id', 'rating', 'timestamp'], dtype='object')

In [14]:
print(movies.shape)
print(ratings.shape)
print(movie_ratings.shape)

(1682, 2)
(100000, 4)
(100000, 5)


##### What if the columns we want to join on don't have the same name?

In [15]:
movies.columns = ['m_id', 'title']
movies.columns

Index(['m_id', 'title'], dtype='object')

In [16]:
ratings.columns

Index(['user_id', 'movie_id', 'rating', 'timestamp'], dtype='object')

In [17]:
# Now try to do the merge

try:
    movie_ratings = pd.merge(movies, ratings)
except Exception as err:
    print(err)
else:
    movie_ratings.head()

No common columns to perform merge on. Merge options: left_on=None, right_on=None, left_index=False, right_index=False


In [18]:
# Here we need to specify the column names of the left table and right table where the merge to take place
# Now once again  try to merge

pd.merge(movies, ratings, left_on='m_id', right_on='movie_id').head()

,m_id,title,user_id,movie_id,rating,timestamp
0,1,Toy Story (1995),308,1,4,887736532
1,1,Toy Story (1995),287,1,5,875334088
2,1,Toy Story (1995),148,1,4,877019411
3,1,Toy Story (1995),280,1,4,891700426
4,1,Toy Story (1995),66,1,3,883601324


##### What if we want to join on one index?

In [19]:
movies = movies.set_index('m_id')
movies.head()

,title
m_id,
1,Toy Story (1995)
2,GoldenEye (1995)
3,Four Rooms (1995)
4,Get Shorty (1995)
5,Copycat (1995)


In [20]:
# When to join on index we pass the parameters 'left_index=True' and "right_on='column name'"
# Here the index of the right dataframe will be used as the index of the result

pd.merge(movies, ratings, left_index=True, right_on='movie_id').head()

,title,user_id,movie_id,rating,timestamp
24,Toy Story (1995),308,1,4,887736532
454,Toy Story (1995),287,1,5,875334088
957,Toy Story (1995),148,1,4,877019411
971,Toy Story (1995),280,1,4,891700426
1324,Toy Story (1995),66,1,3,883601324


##### What if we want to join on two indexes?

In [21]:
ratings = ratings.set_index('movie_id')
ratings.head()

,user_id,rating,timestamp
movie_id,,,
242,196,3,881250949
302,186,3,891717742
377,22,1,878887116
51,244,2,880606923
346,166,1,886397596


In [22]:
# Here we will pass both the index parameter as True
# Here it will take the index of the left dataframe as result
# So index not needed to be unique in Pandas

pd.merge(movies, ratings, left_index=True, right_index=True).head()

,title,user_id,rating,timestamp
1,Toy Story (1995),308,4,887736532
1,Toy Story (1995),287,5,875334088
1,Toy Story (1995),148,4,877019411
1,Toy Story (1995),280,4,891700426
1,Toy Story (1995),66,3,883601324


#### Four Types of Joins

There are actually four types of joins supported by the Pandas merge function. Here's how they are described by the documentation:

- **inner**: use intersection of keys from both frames, similar to a SQL inner join; preserve the order of the left keys
- **outer**: use union of keys from both frames, similar to a SQL full outer join; sort keys lexicographically
- **left**: use only keys from left frame, similar to a SQL left outer join; preserve key order
- **right**: use only keys from right frame, similar to a SQL right outer join; preserve key order

The default is the "**inner join**", which was used when creating the movie_ratings DataFrame.

In [23]:
A = pd.DataFrame({'color': ['green', 'yellow', 'red'], 'num':[1, 2, 3]})
A

,color,num
0,green,1
1,yellow,2
2,red,3


In [24]:
B = pd.DataFrame({'color': ['green', 'yellow', 'pink'], 'size':['S', 'M', 'L']})
B

,color,size
0,green,S
1,yellow,M
2,pink,L


In [25]:
# Inner Join

pd.merge(A, B, how='inner')

,color,num,size
0,green,1,S
1,yellow,2,M


In [26]:
# Outer Join

pd.merge(A, B, how='outer')

,color,num,size
0,green,1.0,S
1,yellow,2.0,M
2,red,3.0,NaN
3,pink,NaN,L


In [27]:
# Left Join

pd.merge(A, B, how='left')

,color,num,size
0,green,1,S
1,yellow,2,M
2,red,3,NaN


In [28]:
# Right Join

pd.merge(A, B, how='right')

,color,num,size
0,green,1.0,S
1,yellow,2.0,M
2,pink,NaN,L
